# Anomaly Detection (Binary 3D)


**Inputs**  


The 14 waveforms (features) reported in the dataset in order are:

- `A+IGBT-I`: Current passing through the IGBT switch of phase A+ in Qa1 ($A$).  
- `A+*IGBT-I`: Current passing through the IGBT switch of phase A+\* in Qa3 ($A$).  
- `B+IGBT-I`: Current passing through the IGBT switch of phase B+ in Qb1 ($A$).  
- `B+*IGBT-I`: Current passing through the IGBT switch of phase B+\* in Qb3 ($A$).  
- `C+IGBT-I`: Current passing through the IGBT switch of phase C+ in Qc1 ($A$).  
- `C+*IGBT-I`: Current passing through the IGBT switch of phase C+\* in Qc3 ($A$).  
- `A-Flux`: Magnetic flux density for phase A in transformer XA ($-$).  
- `B-Flux`: Magnetic flux density for phase B in transformer XB ($-$).  
- `C-Flux`: Magnetic flux density for phase C in transformer XC ($-$).  
- `Mod-V`: Modulator voltage ($V$).  
- `Mod-I`: Modulator current ($A$).  
- `CB-I`: Cap bank current ($-$).  
- `CB-V`: Cap bank voltage ($V$).  
- `DV/DT`: Time derivative change of the Mod\-V voltage ($-$).

**Outputs**
- `Fault`: Faulty wave pulse
- `Normal` Normal wave pulse

![Image](supporting/anomaly/figs/display.jpg)





The files provided in the dataset are:

- `RFQ.npy`: 3D numpy array of the normal and faulty pulses for the RFQ system (shape: (872, 4500, 14)).
- `RFQ_labels.npy`: 2D numpy array of the labels for the RFQ system (shape: (872, 3)).
- `DTL.npy`: 3D numpy array of the normal and faulty pulses for the DTL system (shape: (1077, 4500, 14)).
- `DTL_labels.npy`: 2D numpy array of the labels for the DTL system (shape: (1077, 3)).
- `CCL.npy`: 3D numpy array of the normal and faulty pulses for the CCL system (shape: (2057, 4500, 14)).
- `CCL_labels.npy`: 2D numpy array of the labels for the CCL system (shape: (2057, 3)).
- `SCL.npy`: 3D numpy array of the normal and faulty pulses for the SCL system (shape: (4598, 4500, 14)).
- `SCL_labels.npy`: 2D numpy array of the labels for the SCL system (shape: (4598, 3)).


High Voltage Converter Modulators (HVCM) have frequent failures, making them a major source of down time for the Spallation neutron source. HVCMs are well instrumented to collect large amounts of waveform data including but not limited to modulator current, modulator voltage, magnetic flux, cap bank voltage, cap bank current, and others. These waveform data are collected from all 15 HVCM systems for the application of anomaly detection and failure prediction.

All 15 modulators are grouped into four major subsystems based on their type as follows: 1 modulator as radio-frequency quadrupole (RFQ), 2 modulators as drift-tube linac (DTL), 4 modulators as coupled-cavity linac (CCL), and 8 modulators as super-conducting linac (SCL). A waveform is featured by consecutive pulses, which could be either a normal pulse if the system is healthy, or anomaly pulse if the system is close to failing. These pulses are collected and reported in the dataset of this paper, where the details of the data collection and processing are described in the example.

![Image](supporting/anomaly/figs/data.jpg)

In [2]:
import pickle
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, "supporting/anomaly")

from supporting.anomaly.preprocessing import load_anomaly_data, split_sequences
import supporting.anomaly.settings as anomaly_settings
import pyMAISE as mai

## pyMAISE Initialization
In the settings file, the researchers initialize pyMAISE as a classification problem and load the DTL data set. GPU is defined and utilized for LSTM, GRU, and CNN-LSTM models.

In [3]:
global_settings = mai.init(
    problem_type=anomaly_settings.problem_type,
    random_state=anomaly_settings.random_state,
    cuda_visible_devices="1",  # Use GPU 1
)

The settings file defines a 70/30 training and testing split. To train the machine learning models the researchers desire roughly a 50/50 split between `Fault` and `Normal`. There were significantly more `Normal` cases, so these `normal` datapoints were removed at random points to preserve the sequential pattern of the time dataset.

![Frequency of positive/negative values in anomaly detection data.](supporting/anomaly/figs/bc2_frequency.png)

This plot shows the frequency of `Normal` and `Fault` cases with the trimmed positive data.


In [4]:
data = load_anomaly_data(
    global_settings=global_settings,
    stack_series=True,
    multiclass=False,
    test_size=anomaly_settings.test_size,
    non_faulty_frac=anomaly_settings.non_faulty_frac,
    timestep_step=1,
)
xtrain, xtest, ytrain, ytest = split_sequences(
    data=data[:-1],
    input_steps=10,
    output_steps=1,
    output_position=0,
)

xtrain shape: (303, 4500, 14)
xtest shape: (131, 4500, 14)
ytrain shape: (303, 4500, 2)
ytest shape: (131, 4500, 2)


In [5]:
xtrain

<xarray.DataArray (samples: 1363491, time steps: 10, features: 14)>
array([[[0.60021222, 0.61255407, 0.80077827, ..., 0.50390029,
         0.87899792, 0.50865293],
        [0.59960097, 0.61191434, 0.79995573, ..., 0.50206298,
         0.87865424, 0.50992256],
        [0.59990656, 0.61223418, 0.79913312, ..., 0.50114417,
         0.87876725, 0.51246178],
        ...,
        [0.57973617, 0.60455745, 0.80201221, ..., 0.49884751,
         0.87865424, 0.50738335],
        [0.56353897, 0.60871565, 0.79995573, ..., 0.49884751,
         0.87899792, 0.50738275],
        [0.54734129, 0.61191434, 0.800367  , ..., 0.50068486,
         0.87899792, 0.50738323]],

       [[0.59960097, 0.61191434, 0.79995573, ..., 0.50206298,
         0.87865424, 0.50992256],
        [0.59990656, 0.61223418, 0.79913312, ..., 0.50114417,
         0.87876725, 0.51246178],
        [0.60021222, 0.61287391, 0.800367  , ..., 0.50068486,
         0.87842357, 0.50738335],
...
        [0.60296273, 0.49868202, 0.80160087, ..., 0.50068486,
         0.71915859, 0.51754028],
        [0.60265714, 0.49772298, 0.8011896 , ..., 0.49976614,
         0.71950227, 0.52261567],
        [0.60326833, 0.49804267, 0.80283475, ..., 0.4979288 ,
         0.71893245, 0.51754028]],

       [[0.60296273, 0.49900168, 0.8011896 , ..., 0.49884751,
         0.71950227, 0.52388972],
        [0.60265714, 0.49708104, 0.80077827, ..., 0.4979288 ,
         0.7193892 , 0.52388972],
        [0.60265714, 0.49836233, 0.80201221, ..., 0.49884751,
         0.71915859, 0.52642745],
        ...,
        [0.60265714, 0.49772298, 0.8011896 , ..., 0.49976614,
         0.71950227, 0.52261567],
        [0.60326833, 0.49804267, 0.80283475, ..., 0.4979288 ,
         0.71893245, 0.51754028],
        [0.60265714, 0.49836233, 0.8011896 , ..., 0.49701005,
         0.71904552, 0.52134895]]])
Coordinates:
  * samples     (samples) int64 0 1 2 3 4 ... 1363487 1363488 1363489 1363490
  * time steps  (time steps) int64 0 1 2 3 4 5 6 7 8 9
  * features    (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [6]:
ytrain

<xarray.DataArray (samples: 1363491, features: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])
Coordinates:
  * samples   (samples) int64 0 1 2 3 4 ... 1363487 1363488 1363489 1363490
  * features  (features) object 'Class_Fault' 'Class_Run'

## Model Initialization and Hyperparameter Tuning

The researchers will compare the performance of 3 machine learning (ML) models:

- Long short-term memory: `LSTM`,
- Gated recurrent unit: `GRU`,
- Convolusional neural network: `CNN-LSTM`

Both `LSTM` and `GRU` utilize input, output, and dense input/output layers, in order. The `CNN-LSTM` uses a convolusion, max pooling, and LSTM layers in order. The researchers define the hyperparameter search objective as accuracy, utilizing Bayesian optimization and 50 trials. This is used to find the best hyperparameters that maximize accuracy. The results are saved using `pickle` and imported to Jupyter.

![Convergence of Bayesian optimizer to best hyperparameter configuration.](supporting/anomaly/figs/bc2_convergence.png)

The search could not produce an LSTM/GRU configuration with performance better than ~0.82.

In [8]:
with open("supporting/anomaly/configs/binary_case_2.pkl", "rb") as f:
    configs = pickle.load(f)

## Model Postprocessing

With the models tuned and the top hyperparameters saved, we can now pass these models to the `pyMAISE.PostProcessor` for model comparison and analysis. We will increase the `LSTM`, `GRU`, and `CNN-LSTM` models epochs for better performance.

In [ ]:
postprocessor = mai.PostProcessor(
    data=(xtrain, xtest, ytrain, ytest),
    model_configs=[configs],
    new_model_settings={
        "LSTM": {"fitting_params": {"epochs": 35}},
        "GRU": {"fitting_params": {"epochs": 35}},
        "CNN-LSTM": {"fitting_params": {"epochs": 35}},
    },
)

To compare the performance of these models, we will compute four metrics for both the training and testing data: 
- `Accuracy` = $\frac{\text{Number of Correct Predictions}}{\text{Number of Instances}}$
- `Recall`= $\frac{\text{True Positives}}{\text{True Positives}+\text{False Negatives}}$
- `Precision` = $\frac{\text{True Positives}}{\text{True Positives}+\text{False Positives}}$
- `F1`= $\frac{2*\text{Precision}*\text{Recall}}{\text{Precision}+\text{Recall}}$

In [ ]:
import warnings 
warnings.filterwarnings('ignore') 
postprocessor.metrics()

Based on the performance above, the LSTM model is the top performing model. The top 3 models are LSTM, and their corresponding accuracy scores are in the low 60\% and high 50\% respectively. GRU struggled to produce a higher accuracy than 57%, with an average being around 54%. 

Next, the best-performing model hyperparameters are shown.

In [ ]:
for model in ["LSTM", "GRU", "CNN-LSTM"]:
    for key, value in postprocessor.get_params(model_type=model).to_dict().items():
        print(f"{key}: {value[0]}")
    print()

In [ ]:
postprocessor.confusion_matrix(model_type="LSTM", figsize=(15, 5))
plt.show()

In [ ]:
postprocessor.confusion_matrix(model_type="GRU", figsize=(15, 5))
plt.show()

In [ ]:
postprocessor.confusion_matrix(model_type="CNN-LSTM", figsize=(15, 5))
plt.show()

Confusion matrices show approximately 80% accuracy. The researchers observe more data in the .... 

In [ ]:
postprocessor.nn_learning_plot(model_type="LSTM")
plt.show()

In [ ]:
postprocessor.nn_learning_plot(model_type="GRU")
plt.show()

In [ ]:
postprocessor.nn_learning_plot(model_type="CNN-LSTM")
plt.show()

All validation curves are below the training curve; therefore, shows no overfitting.

![Image](supporting/anomaly/figs/pyMAISElogo.png)